### Příprava datových sad pro dolovací úlohy

In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Dolovací úloha: klasifikace druhů tučňáků na základě ostatních atributů

In [56]:
data = pd.read_csv("penguins_lter.csv")

### Odstranění irelevantních atributů

In [57]:
data = data[["Island", "Culmen Length (mm)", "Culmen Depth (mm)", "Flipper Length (mm)", "Body Mass (g)"]]
data.head()

,Island,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g)
0,Torgersen,39.1,18.7,181.0,3750.0
1,Torgersen,39.5,17.4,186.0,3800.0
2,Torgersen,40.3,18.0,195.0,3250.0
3,Torgersen,NaN,NaN,NaN,NaN
4,Torgersen,36.7,19.3,193.0,3450.0


### Chybějící hodnoty
Využijte alespoň dvě různé metody pro odstranění chybějících hodnot:

1) Ignorování záznamu

2) Doplnění chybějících hodnot - např. průměr, medián, modus, ... (pro celou sadu nebo pro danou třídu)

V první datové sadě budeme záznamy ignorovat, ve druhé doplníme jejich hodnotu průměrem.

In [58]:
# vytvoreni druhe datove sady
data2 = data

In [59]:
# chybejici hodnoty
empty = data[data.isnull().any(1)]
empty

,Island,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g)
3,Torgersen,NaN,NaN,NaN,NaN
339,Biscoe,NaN,NaN,NaN,NaN


In [60]:
# ignorovani zaznamu v prvni sade
data = data[~data.isnull().any(1)]

In [62]:
# nahrazeni chybejici hodnoty prumerem ve druhe sade
cl = data2["Culmen Length (mm)"].to_numpy().reshape(-1,1)
cd = data2["Culmen Depth (mm)"].to_numpy().reshape(-1,1)
fl = data2["Flipper Length (mm)"].to_numpy().reshape(-1,1)
bm = data2["Body Mass (g)"].to_numpy().reshape(-1,1)

data2["Culmen Length (mm)"] = data2["Culmen Length (mm)"].fillna(np.nanmean(cl))
data2["Culmen Depth (mm)"] = data2["Culmen Depth (mm)"].fillna(np.nanmean(cd))
data2["Flipper Length (mm)"] = data2["Flipper Length (mm)"].fillna(np.nanmean(fl))
data2["Body Mass (g)"] = data2["Body Mass (g)"].fillna(np.nanmean(bm))

### Odlehlé hodnoty

In [69]:
def find_outliers(cl):
    # finding the 1st quartile
    q1 = np.quantile(cl, 0.25)
    
    # finding the 3rd quartile
    q3 = np.quantile(cl, 0.75)
    med = np.median(cl)
    
    # finding the iqr region
    iqr = q3-q1
    
    # finding upper and lower whiskers
    upper_bound = q3+(1.5*iqr)
    lower_bound = q1-(1.5*iqr)

    outliers = cl[(cl <= lower_bound) | (cl >= upper_bound)]
    print('The following are the outliers in the boxplot:{}'.format(outliers))

In [70]:
def outliers_for_species(species, data):
    data = data[data["Species"] == species]

    species = data["Species"]
    island = data["Island"]
    cl = data["Culmen Length (mm)"].to_numpy().reshape(1,-1)
    cd = data["Culmen Depth (mm)"].to_numpy().reshape(-1,1)
    fl = data["Flipper Length (mm)"].to_numpy().reshape(-1,1)
    bm = data["Body Mass (g)"].to_numpy().reshape(-1,1)

    find_outliers(cl)
    find_outliers(cd)
    find_outliers(fl)
    find_outliers(bm)